In [ ]:
!pip install torch

In [ ]:
!pip install torchvision

In [ ]:
!pip install utils

In [13]:
!pip install python-utils

In [10]:
!pip install tqdm  # or another package if necessary

In [14]:
from utils import progress_bar, simple_FC

ImportError: cannot import name 'progress_bar' from 'utils' (/opt/conda/lib/python3.12/site-packages/utils/__init__.py)

# Code du papier brut

In [15]:
'''Train MNIST with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import re
import argparse
import numpy as np

#from utils import progress_bar
from utils import simple_FC

import matplotlib.pyplot as plt

def train(epoch, net):
    if args.verbose:
        print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        targets = torch.nn.functional.one_hot(targets, num_classes=10).float()
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets.argmax(1)).sum().item()
        if args.verbose:
            progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return 100.*correct/total, train_loss/(batch_idx+1)


def test(epoch, net, model_name, save_checkpoint=False):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            targets = torch.nn.functional.one_hot(targets, num_classes=10).float()
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets.argmax(1)).sum().item()
            if args.verbose:
                progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                             % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    acc = 100.*correct/total
    if save_checkpoint:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        torch.save(state, os.path.join(args.ckpt_path, '%s.pth'%model_name))
        best_acc = acc
    return 100.*correct/total, test_loss/(batch_idx+1)


parser = argparse.ArgumentParser(description='PyTorch MNIST Double Descent Curve')
parser.add_argument('--verbose', default=0, type=int, help='level of verbos')
parser.add_argument('--reuse', action='store_true', help='parameter reuse')
parser.add_argument('--data_path', default='./data', type=str, help='data directory')
parser.add_argument('--ckpt_path', default='./ckpt', type=str, help='checkpoint directory')
parser.add_argument('--log_path', default='./log', type=str, help='log directory')
args = parser.parse_args()

if not os.path.isdir(args.data_path):
    os.mkdir(args.data_path)
if not os.path.isdir(args.ckpt_path):
    os.mkdir(args.ckpt_path)
if not os.path.isdir(args.log_path):
    os.mkdir(args.log_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data: no data augmentation is used
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.MNIST(
    root=args.data_path, train=True, download=True, transform=transform_train)
trainset = torch.utils.data.Subset(trainset, indices=np.arange(4000))
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(
    root=args.data_path, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

n_hidden_units = [1, 3, 5, 7, 9, 10, 20, 30, 40, 45, 47, 49, 50, 51, 53, 55, 60, 70, 80, 90, 100, 110, 130, 150, 170, 200, 250]
n_epoch = 6000

for n_hidden_unit in n_hidden_units:
    # Model
    net = simple_FC(n_hidden_unit)
    print('Number of parameters: %d'%sum(p.numel() for p in net.parameters()))
    net = net.cuda()
    net = net.to(device)
    if device == 'cuda':
        net = net.cuda()
        cudnn.benchmark = True
    ### initialization
    if n_hidden_unit == 1: # smallest network
        torch.nn.init.xavier_uniform_(net.features[1].weight, gain=1.0)
        torch.nn.init.xavier_uniform_(net.classifier.weight, gain=1.0)
    elif n_hidden_unit > 50: # interpolation point: Number of data (4000) * number of class (10) = number of parameters (50*784 + 50 + 50*10 + 10)
        torch.nn.init.normal_(net.features[1].weight, mean=0.0, std=0.1)
        torch.nn.init.normal_(net.classifier.weight, mean=0.0, std=0.1)
    else: 
        torch.nn.init.normal_(net.features[1].weight, mean=0.0, std=0.1)
        torch.nn.init.normal_(net.classifier.weight, mean=0.0, std=0.1)
        if args.reuse:
            print('use previous checkpoints to initialize the weights')
            i = 1 # load the closest previous model for weight reuse
            while not os.path.exists(os.path.join(args.ckpt_path, 'simple_FC_%d.pth'%(n_hidden_unit-i))):
                print('loading from simple_FC_%d.pth'%(n_hidden_unit-i))
                i += 1
            checkpoint = torch.load(os.path.join(args.ckpt_path, 'simple_FC_%d.pth'%(n_hidden_unit-i)))
            with torch.no_grad():
                net.features[1].weight[:n_hidden_unit-i, :].copy_(checkpoint['net']['features.1.weight'])
                net.features[1].bias[:n_hidden_unit-i].copy_(checkpoint['net']['features.1.bias'])
                net.classifier.weight[:, :n_hidden_unit-i].copy_(checkpoint['net']['classifier.weight'])
                net.classifier.bias.copy_(checkpoint['net']['classifier.bias'])
    ### training and testing
    best_acc = 0
    start_epoch = 0
    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.95)
    for epoch in range(start_epoch, start_epoch+n_epoch):
        if (epoch+1) % 500 == 0:
            if n_hidden_unit <= 50: # learning rate schedule
                optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.9
        train_acc, train_loss = train(epoch, net)
        if n_hidden_unit <= 50 and train_acc == 1 or epoch == start_epoch+n_epoch-1: # early stop before interpolation
            test_acc, test_loss = test(epoch, net, 'simple_FC_%d'%(n_hidden_unit), save_checkpoint=True)
            print('classification error reaches 0, stop training')
            break
    print('Training Loss: %.3f | Acc: %.3f%%' % (train_loss, train_acc))
    print('Test Loss: %.3f | Acc: %.3f%%\n' % (test_loss, test_acc))
    with open(os.path.join(args.log_path, 'FC_%d.txt'%n_hidden_unit), 'w') as fw:
        fw.write('Number of parameters: %d\n'%sum(p.numel() for p in net.parameters()))
        fw.write('Training Loss: %.3f | Acc: %.3f%%\n' % (train_loss, train_acc))
        fw.write('Test Loss: %.3f | Acc: %.3f%%\n' % (test_loss, test_acc))



model_names = sorted([int(fn.split('_')[1].split('.')[0]) for fn in os.listdir(args.log_path)])

train_losses = {model_name:0. for model_name in model_names}
test_losses = {model_name:0. for model_name in model_names}
train_accs = {model_name:0. for model_name in model_names}
test_accs = {model_name:0. for model_name in model_names}
n_params = {model_name:0. for model_name in model_names}

for model_name in model_names:
    with open(os.path.join('log', 'FC_%d.txt'%(model_name))) as f:
        for line in f:
            if line.startswith('Number'):
                n_params[model_name] = float(line.rstrip().split()[-1])
            if line.startswith('Training'):
                loss = re.search(r'Loss: (.*?) \|', line).group(1)
                train_losses[model_name] = float(loss)
                acc = re.search(r'Acc: (.*?)\%', line).group(1)
                train_accs[model_name] = float(acc)
            if line.startswith('Test'):
                loss = re.search(r'Loss: (.*?) \|', line).group(1)
                test_losses[model_name] = float(loss)
                acc = re.search(r'Acc: (.*?)\%', line).group(1)
                test_accs[model_name] = float(acc)


# plot 
plt.clf()
fig = plt.figure()
ax = plt.subplot(111)
plt.plot([n_params[model_name] for model_name in model_names], [train_losses[model_name] for model_name in model_names], marker='o', label='train', color='#e31a1c')
plt.plot([n_params[model_name] for model_name in model_names], [test_losses[model_name] for model_name in model_names], marker='o', label='test', color='#1f78b4')
plt.ylabel('loss')
box = ax.get_position()
plt.tight_layout()
ax.set_position([box.x0, box.y0,
             box.width, box.height * 0.9])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.25), fancybox=True, ncol=4)
if args.reuse:
    plt.savefig('MNIST_double_descent_loss_w_weight_reuse.png')
else:
    plt.savefig('MNIST_double_descent_loss_wo_weight_reuse.png')

plt.clf()
fig = plt.figure()
ax = plt.subplot(111)
plt.plot([n_params[model_name] for model_name in model_names], [train_accs[model_name] for model_name in model_names], marker='o', label='train', color='#e31a1c')
plt.plot([n_params[model_name] for model_name in model_names], [test_accs[model_name] for model_name in model_names], marker='o', label='test', color='#1f78b4')
plt.ylabel('accuracy')
box = ax.get_position()
plt.tight_layout()
ax.set_position([box.x0, box.y0,
             box.width, box.height * 0.9])
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.25), fancybox=True, ncol=4)
if args.reuse:
    plt.savefig('MNIST_double_descent_accuracy_w_weight_reuse.png')
else:
    plt.savefig('MNIST_double_descent_accuracy_wo_weight_reuse.png')

ImportError: cannot import name 'simple_FC' from 'utils' (/opt/conda/lib/python3.12/site-packages/utils/__init__.py)

# Ajout commentaires sur code

In [ ]:
'''Train MNIST with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import re
import argparse
import numpy as np

from utils import progress_bar, simple_FC

import matplotlib.pyplot as plt




# Fonction d'entraînement d'un modèle pour une époque donnée
def train(epoch, net):
    # Si 'verbose' est activé, afficher l'indice de l'époque en cours
    if args.verbose:
        print('\nEpoch: %d' % epoch)

    # Passer le réseau en mode entraînement (cela active des comportements comme la régularisation par dropout)
    net.train()

    # Initialisation des variables pour suivre la perte (loss) et l'exactitude (accuracy) pendant l'entraînement
    train_loss = 0  # Somme des pertes (loss) pour la période
    correct = 0      # Nombre de prédictions correctes
    total = 0        # Nombre total de données traitées

    # Boucle sur les mini-batches du DataLoader d'entraînement
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        # Conversion des labels (cibles) en encodage one-hot et en type float
        targets = torch.nn.functional.one_hot(targets, num_classes=10).float()
        
        # Transférer les entrées et les cibles sur le bon périphérique (CPU ou GPU)
        inputs, targets = inputs.to(device), targets.to(device)

        # Réinitialiser les gradients des paramètres du modèle avant le calcul
        optimizer.zero_grad()

        # Effectuer une passe avant dans le réseau pour obtenir les prédictions
        outputs = net(inputs)

        # Calcul de la perte entre les sorties du modèle et les cibles (labels)
        loss = criterion(outputs, targets)

        # Rétropropagation de la perte pour calculer les gradients
        loss.backward()

        # Mise à jour des poids du modèle en fonction des gradients
        optimizer.step()

        # Accumuler la perte totale de l'entraînement
        train_loss += loss.item()

        # Trouver l'indice de la classe prédite (la classe avec la plus grande probabilité)
        _, predicted = outputs.max(1)

        # Ajouter le nombre total d'exemples traités dans ce mini-batch
        total += targets.size(0)

        # Comparer les prédictions avec les cibles pour calculer le nombre de prédictions correctes
        # `predicted.eq(targets.argmax(1))` renvoie un vecteur booléen où True indique que la prédiction est correcte
        correct += predicted.eq(targets.argmax(1)).sum().item()

        # Si 'verbose' est activé, afficher une barre de progression avec la perte moyenne et la précision actuelle
        if args.verbose:
            progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Retourner l'exactitude en pourcentage et la perte moyenne sur l'époque
    return 100.*correct/total, train_loss/(batch_idx+1)









def test(epoch, net, model_name, save_checkpoint=False):
    # Déclare une variable globale 'best_acc' qui est utilisée pour suivre la meilleure précision obtenue sur les tests.
    global best_acc
    
    # Place le réseau de neurones en mode évaluation (inhibe la mise à jour des gradients et les mécanismes comme Dropout et BatchNorm)
    net.eval()

    # Initialisation de variables pour suivre la perte de test et la précision
    test_loss = 0  # Somme des pertes pour toutes les étapes de test
    correct = 0     # Nombre de prédictions correctes
    total = 0       # Nombre total d'exemples traités

    # Utilisation de 'torch.no_grad()' pour indiquer que les gradients ne seront pas calculés durant la phase de test
    # Cela permet de réduire la mémoire utilisée et d'accélérer les calculs
    with torch.no_grad():
        # Itération sur le DataLoader de test (chaque itération correspond à un lot de données)
        for batch_idx, (inputs, targets) in enumerate(testloader):
            # Conversion des étiquettes de classe en représentation one-hot (chaque étiquette devient un vecteur binaire)
            # 'num_classes=10' spécifie qu'il y a 10 classes pour la classification
            targets = torch.nn.functional.one_hot(targets, num_classes=10).float()

            # Déplacement des données vers le bon appareil (GPU ou CPU)
            inputs, targets = inputs.to(device), targets.to(device)

            # Passage des entrées dans le modèle pour obtenir les prédictions
            outputs = net(inputs)

            # Calcul de la perte entre les sorties du modèle et les étiquettes cibles
            loss = criterion(outputs, targets)

            # Ajout de la perte de ce lot au total de la perte de test
            test_loss += loss.item()

            # Prédiction de la classe (index de la classe avec la probabilité la plus élevée)
            _, predicted = outputs.max(1)

            # Mise à jour du nombre total d'exemples traités
            total += targets.size(0)

            # Mise à jour du nombre de prédictions correctes
            correct += predicted.eq(targets.argmax(1)).sum().item()

            # Affichage de l'état d'avancement du test si l'option verbose est activée
            if args.verbose:
                # Affiche la perte moyenne et la précision jusqu'à l'instant
                progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                             % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Calcul de la précision en pourcentage
    acc = 100. * correct / total

    # Si l'option 'save_checkpoint' est activée, on enregistre l'état du modèle
    if save_checkpoint:
        print('Saving..')
        # Création de l'état du modèle à sauvegarder
        state = {
            'net': net.state_dict(),   # Poids du modèle (état du réseau de neurones)
            'acc': acc,                 # Précision sur l'ensemble de test
            'epoch': epoch,             # Numéro de l'époque actuelle
        }
        # Sauvegarde de l'état dans un fichier .pth (format PyTorch)
        torch.save(state, os.path.join(args.ckpt_path, '%s.pth'%model_name))
        
        # Mise à jour de la meilleure précision observée
        best_acc = acc

    # Retourne la précision en pourcentage et la perte moyenne pour l'ensemble de test
    return 100. * correct / total, test_loss / (batch_idx + 1)













# Création d'un parseur d'arguments pour la ligne de commande
parser = argparse.ArgumentParser(description='PyTorch MNIST Double Descent Curve')

# Ajout d'un argument pour définir le niveau de verbosité (affichage des détails)
parser.add_argument('--verbose', default=0, type=int, help='level of verbosity')

# Ajout d'un argument pour permettre la réutilisation des paramètres (option booléenne)
parser.add_argument('--reuse', action='store_true', help='parameter reuse')

# Spécification du répertoire des données d'entrée (dossier où les données seront lues)
parser.add_argument('--data_path', default='./data', type=str, help='data directory')

# Spécification du répertoire pour les modèles sauvegardés (checkpoints)
parser.add_argument('--ckpt_path', default='./ckpt', type=str, help='checkpoint directory')

# Spécification du répertoire pour les fichiers de logs
parser.add_argument('--log_path', default='./log', type=str, help='log directory')

# Analyse des arguments passés à la ligne de commande
args = parser.parse_args()

# Vérification et création des répertoires nécessaires si ils n'existent pas déjà

# Si le répertoire pour les données n'existe pas, on le crée
if not os.path.isdir(args.data_path):
    os.mkdir(args.data_path)

# Si le répertoire pour les checkpoints n'existe pas, on le crée
if not os.path.isdir(args.ckpt_path):
    os.mkdir(args.ckpt_path)

# Si le répertoire pour les logs n'existe pas, on le crée
if not os.path.isdir(args.log_path):
    os.mkdir(args.log_path)

# Détermination du périphérique utilisé pour l'exécution : GPU si disponible, sinon CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Préparation des données
# Affichage d'un message indiquant que les données sont en train de se préparer
print('==> Preparing data..')

# Définition des transformations à appliquer sur les images d'entraînement
transform_train = transforms.Compose([
    # Transformation des images en tenseurs
    transforms.ToTensor(),
])

# Définition des transformations à appliquer sur les images de test
transform_test = transforms.Compose([
    # Transformation des images en tenseurs
    transforms.ToTensor(),
])

# Chargement du jeu de données MNIST pour l'entraînement
trainset = torchvision.datasets.MNIST(
    root=args.data_path,     # Chemin vers les données, défini précédemment dans les arguments
    train=True,              # Indique qu'il s'agit de la partition d'entraînement (True pour entraînement, False pour test)
    download=True,           # Si les données ne sont pas présentes localement, elles seront téléchargées
    transform=transform_train  # Transformation à appliquer aux images d'entraînement (convertir en tenseur)
)

# Création d'un sous-ensemble du jeu de données d'entraînement en ne prenant que les 4000 premiers exemples
trainset = torch.utils.data.Subset(trainset, indices=np.arange(4000))

# Création d'un DataLoader pour charger les données d'entraînement par lot
trainloader = torch.utils.data.DataLoader(
    trainset,                # Le sous-ensemble des données d'entraînement
    batch_size=128,          # Taille de chaque lot (128 images par lot)
    shuffle=True,            # Mélange des données avant chaque époque pour éviter les biais dans l'entraînement
    num_workers=2            # Nombre de processus parallèles pour charger les données (accélère le chargement)
)

# Chargement du jeu de données MNIST pour le test
testset = torchvision.datasets.MNIST(
    root=args.data_path,     # Chemin vers les données
    train=False,             # Indique qu'il s'agit de la partition de test
    download=True,           # Si les données ne sont pas présentes localement, elles seront téléchargées
    transform=transform_test  # Transformation à appliquer aux images de test (convertir en tenseur)
)

# Création d'un DataLoader pour charger les données de test par lot
testloader = torch.utils.data.DataLoader(
    testset,                 # Jeu de données de test
    batch_size=100,          # Taille de chaque lot (100 images par lot)
    shuffle=False,           # Pas besoin de mélanger les données de test, car elles sont utilisées pour évaluation
    num_workers=2            # Nombre de processus parallèles pour charger les données
)

# Liste des tailles de couches cachées à tester dans l'architecture du modèle
n_hidden_units = [1, 3, 5, 7, 9, 10, 20, 30, 40, 45, 47, 49, 50, 51, 53, 55, 60, 70, 80, 90, 
                  100, 110, 130, 150, 170, 200, 250]

# Nombre d'époques d'entraînement (6000 époques)
n_epoch = 6000

# Boucle pour tester différentes tailles de couches cachées
for n_hidden_unit in n_hidden_units:
    # Création du modèle (réseau de neurones) avec le nombre de neurones spécifié dans n_hidden_unit
    net = simple_FC(n_hidden_unit)  # simple_FC est une fonction qui génère un réseau avec n_hidden_unit neurones cachés
    # Affichage du nombre total de paramètres du modèle
    print('Number of parameters: %d' % sum(p.numel() for p in net.parameters()))
    
    # Envoi du modèle sur le périphérique (GPU ou CPU)
    net = net.cuda()  # Envoie le modèle sur le GPU si disponible (CUDA)
    net = net.to(device)  # Envoie le modèle vers le périphérique 'cuda' ou 'cpu', selon le cas
    
    # Configuration spécifique pour les GPU si CUDA est disponible
    if device == 'cuda':
        net = net.cuda()  # Envoie le modèle sur le GPU
        cudnn.benchmark = True  # Active l'optimisation pour les architectures de GPU avec des tailles d'entrée fixes
    
    ### Initialisation des poids du modèle ###
    if n_hidden_unit == 1:  # Si le réseau a 1 neurone caché, c'est le plus petit réseau
        # Initialisation des poids avec la méthode Xavier (uniforme) pour les couches cachées et la couche de sortie
        torch.nn.init.xavier_uniform_(net.features[1].weight, gain=1.0)
        torch.nn.init.xavier_uniform_(net.classifier.weight, gain=1.0)
    elif n_hidden_unit > 50:  # Si le nombre de neurones cachés est supérieur à 50, on utilise une initialisation normale
        # Initialisation des poids avec une distribution normale (moyenne=0, écart-type=0.1) pour éviter des valeurs extrêmes
        torch.nn.init.normal_(net.features[1].weight, mean=0.0, std=0.1)
        torch.nn.init.normal_(net.classifier.weight, mean=0.0, std=0.1)
    else:  # Si le nombre de neurones cachés est entre 1 et 50, on applique une initialisation normale
        torch.nn.init.normal_(net.features[1].weight, mean=0.0, std=0.1)
        torch.nn.init.normal_(net.classifier.weight, mean=0.0, std=0.1)
        
        # Si l'argument --reuse est passé, on réutilise les poids d'un modèle précédent pour l'initialisation
        if args.reuse:
            print('use previous checkpoints to initialize the weights')
            i = 1  # On commence avec le modèle précédent le plus proche
            while not os.path.exists(os.path.join(args.ckpt_path, 'simple_FC_%d.pth' % (n_hidden_unit - i))):
                print('loading from simple_FC_%d.pth' % (n_hidden_unit - i))  # Chargement du modèle précédent
                i += 1  # On cherche de plus en plus en arrière dans les checkpoints
            # Chargement du checkpoint correspondant
            checkpoint = torch.load(os.path.join(args.ckpt_path, 'simple_FC_%d.pth' % (n_hidden_unit - i)))
            with torch.no_grad():
                # Copie des poids du modèle préexistant dans le modèle actuel
                net.features[1].weight[:n_hidden_unit - i, :].copy_(checkpoint['net']['features.1.weight'])
                net.features[1].bias[:n_hidden_unit - i].copy_(checkpoint['net']['features.1.bias'])
                net.classifier.weight[:, :n_hidden_unit - i].copy_(checkpoint['net']['classifier.weight'])
                net.classifier.bias.copy_(checkpoint['net']['classifier.bias'])
    
    ### Entraînement et test du modèle ###
    best_acc = 0  # Initialisation de la meilleure précision obtenue
    start_epoch = 0  # Début de l'entraînement à l'époque 0
    criterion = nn.MSELoss()  # Fonction de perte, ici l'erreur quadratique moyenne (MSE)
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.95)  # Optimiseur SGD avec un taux d'apprentissage de 0.01 et momentum de 0.95
    
    # Boucle d'entraînement
    for epoch in range(start_epoch, start_epoch + n_epoch):
        # Mise à jour du taux d'apprentissage à chaque 500 itérations
        if (epoch + 1) % 500 == 0:
            if n_hidden_unit <= 50:  # Ajustement du taux d'apprentissage pour les petits réseaux
                optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.9  # Réduit le taux d'apprentissage de 10%
        
        # Entraînement sur l'époque courante
        train_acc, train_loss = train(epoch, net)
        
        # Condition d'arrêt anticipé (early stopping) : arrêt si la précision d'entraînement atteint 100% ou si on atteint la dernière époque
        if n_hidden_unit <= 50 and train_acc == 1 or epoch == start_epoch + n_epoch - 1:
            # Test du modèle et sauvegarde des poids
            test_acc, test_loss = test(epoch, net, 'simple_FC_%d' % (n_hidden_unit), save_checkpoint=True)
            print('classification error reaches 0, stop training')  # Affichage du message d'arrêt
            break
    
    # Affichage des résultats de l'entraînement et du test
    print('Training Loss: %.3f | Acc: %.3f%%' % (train_loss, train_acc))
    print('Test Loss: %.3f | Acc: %.3f%%\n' % (test_loss, test_acc))
    
    # Sauvegarde des résultats dans un fichier texte
    with open(os.path.join(args.log_path, 'FC_%d.txt' % n_hidden_unit), 'w') as fw:
        # Écriture du nombre de paramètres du modèle
        fw.write('Number of parameters: %d\n' % sum(p.numel() for p in net.parameters()))
        # Écriture des résultats d'entraînement
        fw.write('Training Loss: %.3f | Acc: %.3f%%\n' % (train_loss, train_acc))
        # Écriture des résultats de test
        fw.write('Test Loss: %.3f | Acc: %.3f%%\n' % (test_loss, test_acc))




# Récupération et tri des noms des modèles à partir des fichiers dans le dossier de log
model_names = sorted([int(fn.split('_')[1].split('.')[0]) for fn in os.listdir(args.log_path)])

# Initialisation des dictionnaires pour stocker les pertes (losses), les précisions (accs), et le nombre de paramètres pour chaque modèle
train_losses = {model_name: 0. for model_name in model_names}
test_losses = {model_name: 0. for model_name in model_names}
train_accs = {model_name: 0. for model_name in model_names}
test_accs = {model_name: 0. for model_name in model_names}
n_params = {model_name: 0. for model_name in model_names}


# Pour chaque modèle, ouvrir son fichier log et extraire les informations de performance et les paramètres
for model_name in model_names:
    with open(os.path.join('log', 'FC_%d.txt' % (model_name))) as f:
        for line in f:
            if line.startswith('Number'):
                # Si la ligne commence par 'Number', c'est la ligne qui contient le nombre de paramètres du modèle
                n_params[model_name] = float(line.rstrip().split()[-1])  # Extraire le dernier élément de la ligne (nombre de paramètres)
            
            if line.startswith('Training'):
                # Si la ligne commence par 'Training', c'est une ligne de perte et précision pendant l'entraînement
                loss = re.search(r'Loss: (.*?) \|', line).group(1)  # Utilisation de regex pour extraire la perte
                train_losses[model_name] = float(loss)  # Stocker la perte d'entraînement dans le dictionnaire
                acc = re.search(r'Acc: (.*?)\%', line).group(1)  # Utilisation de regex pour extraire la précision
                train_accs[model_name] = float(acc)  # Stocker la précision d'entraînement dans le dictionnaire

            if line.startswith('Test'):
                # Si la ligne commence par 'Test', c'est une ligne de perte et précision pendant le test
                loss = re.search(r'Loss: (.*?) \|', line).group(1)  # Utilisation de regex pour extraire la perte
                test_losses[model_name] = float(loss)  # Stocker la perte de test dans le dictionnaire
                acc = re.search(r'Acc: (.*?)\%', line).group(1)  # Utilisation de regex pour extraire la précision
                test_accs[model_name] = float(acc)  # Stocker la précision de test dans le dictionnaire



# Nettoyage et préparation d'un nouveau graphique pour afficher la perte (loss) en fonction du nombre de paramètres
plt.clf()  # Efface la figure précédente, si elle existe
fig = plt.figure()  # Crée une nouvelle figure
ax = plt.subplot(111)  # Crée un sous-graphe (axes) dans la figure (1 ligne, 1 colonne, 1ère position)

# Tracé de la courbe de perte d'entraînement en fonction du nombre de paramètres
plt.plot([n_params[model_name] for model_name in model_names], 
         [train_losses[model_name] for model_name in model_names], 
         marker='o', label='train', color='#e31a1c')
# Tracé de la courbe de perte de test en fonction du nombre de paramètres
plt.plot([n_params[model_name] for model_name in model_names], 
         [test_losses[model_name] for model_name in model_names], 
         marker='o', label='test', color='#1f78b4')

# Ajout d'un label à l'axe des ordonnées (loss)
plt.ylabel('loss')

# Ajuste la mise en page du graphique pour éviter que les éléments ne se chevauchent
box = ax.get_position()  # Récupère la position actuelle du sous-graphe (axes)
plt.tight_layout()  # Ajuste automatiquement les paramètres de la figure pour éviter que les éléments ne se chevauchent
ax.set_position([box.x0, box.y0, box.width, box.height * 0.9])  # Réduit la hauteur du sous-graphe (axes) pour faire de la place à la légende

# Ajout d'une légende en haut du graphique, avec des options de mise en forme
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.25), fancybox=True, ncol=4)

# Enregistrement du graphique dans un fichier, en fonction de l'argument 'reuse'
if args.reuse:
    plt.savefig('MNIST_double_descent_loss_w_weight_reuse.png')  # Sauvegarde le graphique avec le titre adapté si 'reuse' est activé
else:
    plt.savefig('MNIST_double_descent_loss_wo_weight_reuse.png')  # Sauvegarde le graphique sans réutilisation des poids

# Nettoyage et préparation d'un nouveau graphique pour afficher la précision (accuracy) en fonction du nombre de paramètres
plt.clf()  # Efface la figure précédente
fig = plt.figure()  # Crée une nouvelle figure
ax = plt.subplot(111)  # Crée un sous-graphe (axes) dans la figure (1 ligne, 1 colonne, 1ère position)

# Tracé de la courbe de précision d'entraînement en fonction du nombre de paramètres
plt.plot([n_params[model_name] for model_name in model_names], 
         [train_accs[model_name] for model_name in model_names], 
         marker='o', label='train', color='#e31a1c')
# Tracé de la courbe de précision de test en fonction du nombre de paramètres
plt.plot([n_params[model_name] for model_name in model_names], 
         [test_accs[model_name] for model_name in model_names], 
         marker='o', label='test', color='#1f78b4')

# Ajout d'un label à l'axe des ordonnées (accuracy)
plt.ylabel('accuracy')

# Ajuste la mise en page du graphique pour éviter que les éléments ne se chevauchent
box = ax.get_position()  # Récupère la position actuelle du sous-graphe (axes)
plt.tight_layout()  # Ajuste automatiquement les paramètres de la figure
ax.set_position([box.x0, box.y0, box.width, box.height * 0.9])  # Réduit la hauteur pour faire de la place à la légende

# Ajout d'une légende en haut du graphique, avec des options de mise en forme
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.25), fancybox=True, ncol=4)

# Enregistrement du graphique dans un fichier, en fonction de l'argument 'reuse'
if args.reuse:
    plt.savefig('MNIST_double_descent_accuracy_w_weight_reuse.png')  # Sauvegarde du graphique avec réutilisation des poids
else:
    plt.savefig('MNIST_double_descent_accuracy_wo_weight_reuse.png')  # Sauvegarde du graphique sans réutilisation des poids
